# Simple Regression

First we load the data

In [8]:
import pandas as pd

# Load the dataset
df = pd.read_pickle("../data/winequality.pkl")

In [9]:
import numpy as np

def gradient_descent(X, y, learning_rate=0.01, n_iterations=1000):
    m = len(y)
    theta = np.zeros(X.shape[1])
    for _ in range(n_iterations):
        gradients = 2/m * X.T.dot(X.dot(theta) - y)
        theta -= learning_rate * gradients
    return theta

In [10]:
# X = chlorides, y = quality
X = df['chlorides'].values.reshape(-1, 1)
y = df['quality'].values.reshape(-1, 1)

m = len(y)

# Add bias term (column of ones) for intercept
X_b = np.hstack([np.ones((m, 1)), X])  # shape (m, 2)


theta = gradient_descent(X_b, y)

print("Theta (intercept, slope):")
print(theta)

ValueError: operands could not be broadcast together with shapes (2,) (2,1143) (2,) 

In [ ]:
# X = alcohol, y = quality
X = df['alcohol'].values.reshape(-1, 1)
y = df['quality'].values.reshape(-1, 1)

m = len(y)

# Add bias term (column of ones) for intercept
X_b = np.hstack([np.ones((m, 1)), X])  # shape (m, 2)

theta = gradient_descent(X_b, y.flatten(), learning_rate=0.001)

print("Theta (intercept, slope):")
print(theta)